# 7 Sensitivity 

This notebook presents computations in section 7 of th paper.

To check the previous section:

[Section 6: A climate component of a social planner's decision problem](sec6_DecisionProblem.ipynb)

Next section:

[Section 8: Uncertainty decomposition](sec8_UncertaintyDecomposition.ipynb)


texts in section 7


In this section, we focus on brownian misspecification and jump misspecification.
Therefore, in the following analysis and computations, uncertainty parameter for ambiguity aversion, $\xi_a$ is set to $\infty$. 

## 7.1 Robust perspective

Continue with the baseline probabilities of damage functions $(\cfrac13, \cfrac13, \cfrac13)$.

We start by imposing $\xi_b = \xi_p = 5$, and also explore other sets of parameters $(\xi_b, \xi_p) \in \{(\infty, \infty), (0.3,5),(0.3,0.3)\}$.

In [23]:
%matplotlib inline
import pandas as pd
import numpy as np
from source.model import solve_hjb_y, solve_hjb_y_jump
from source.utilities import find_nearest_value
from source.simulation import simulate_jump
import plotly.graph_objects as go

In [2]:
# ambiguity aversion
ξ_a = 100_000

# Preference
η = .032
δ = .01

# Climate sensitivity
θ_list = pd.read_csv('data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ_list)/len(θ_list)

# Damage function
σ_y = 1.2*np.mean(θ_list)
y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
γ_2p = np.array([0, .0197*2, .3853*2])
πd_o = np.array([1./3, 1./3, 1./3])

y_step = .02
y_grid_long = np.arange(0., 4., y_step)
y_grid_short = np.arange(0., y_bar+y_step, y_step)
n_bar = find_nearest_value(y_grid_long, y_bar) + 1

# Prepare ϕ conditional on low, high, extreme damage
solution = dict()
params_list = [[100_000, 100_000], [5, 5], [0.3, 5], [0.3, 0.3]]
for ξ_b_i, ξ_p_i in params_list:
    model_res_list = []
    for γ_2p_i in γ_2p:
        model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p_i, θ_list, πc_o, ξ_b_i, ξ_a) 
        model_res = solve_hjb_y(y_grid_long, model_args, v0=None, ϵ=1.,
                                tol=1e-8, max_iter=5_000, print_iteration=False)
        model_res_list.append(model_res)

    ϕ_list = [res['v'] for res in model_res_list]
    certainty_equivalent = -ξ_p_i*np.log(np.average(np.exp(-1./ξ_p_i*np.array(ϕ_list)), axis=0, weights=πd_o))
    # Change grid from 0-4 to 0-2
    ϕ_i = np.array([temp[:n_bar] for temp in ϕ_list])
    # Compute ϕ with jump (impose boundary condition)
    model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p, θ_list, πc_o, ϕ_i, πd_o, ξ_b_i, ξ_p_i, ξ_a)
    model_res = solve_hjb_y_jump(y_grid_short, model_args, 
                             v0=np.average(ϕ_i, weights=πd_o, axis=0),
                             ϵ=1., tol=1e-8, max_iter=5_000, print_iteration=False)
    simulation_res = simulate_jump(model_res, θ_list)
    solution[(ξ_b_i, ξ_p_i)] = dict(
        model_res=model_res, 
        simulation_res=simulation_res,
        ξ_b=ξ_b_i,
        ξ_p=ξ_p_i
    )

Converged. Total iteration 5000: LHS Error: 3.581810010899744e-05; RHS Error 0.00020298652865735622
Converged. Total iteration 1502: LHS Error: 9.798189593013262e-09; RHS Error 0.000482581355743833
Converged. Total iteration 1618: LHS Error: 9.993458505164199e-09; RHS Error 0.0022617538835780562
Converged. Total iteration 494: LHS Error: 8.617897950813358e-09; RHS Error 0.0006257523076260985
Converged. Total iteration 5000: LHS Error: 1.826122455561574e-05; RHS Error 7.387945382480998e-05
Converged. Total iteration 1499: LHS Error: 9.937083822464388e-09; RHS Error 0.0004770745076004973
Converged. Total iteration 1620: LHS Error: 9.887198615388115e-09; RHS Error 0.0022398810398579763
Converged. Total iteration 446: LHS Error: 8.221708647226933e-09; RHS Error 0.0008407372192420631
Converged. Total iteration 1461: LHS Error: 9.95181270724288e-09; RHS Error 8.624535502448072e-05
Converged. Total iteration 1506: LHS Error: 9.910680942581962e-09; RHS Error 0.00041269360735776
Converged. Tota

In [24]:
# distorted probabilities
fig = go.Figure()
for ξ_b_i, ξ_p_i in params_list:
    πd = solution[(ξ_b_i, ξ_p_i)]['model_res']['πd']
    for πd_i, name, color in zip(πd,['low', 'high', 'extreme'], ['darkgreen', 'darkorange', 'red']):
        fig.add_trace(go.Scatter(x=y_grid_short, y=πd_i, 
                                 visible=False,
                                 line=dict(color=color, width=2.5),
                                 name=name+" damage"
                                )) 


num_lines = len(πd)
for i in range(num_lines):
    fig.data[1*num_lines + i].visible = True

steps = []
for i in range(len(params_list)):
    # Hide all traces
    label = r"ξb = {:.2f}, ξp = {:.2f}".format(params_list[i][0],params_list[i][1])
    step = dict(
        method ='update',  
        args = [{'visible': [False] * len(fig.data)},
               {"title": "${}\\\ ξ_b = {:.2f},\quad ξ_p = {:.2f}$".format(
                   "Distorted\ probability", 
                   params_list[i][0],
                   params_list[i][1]
               )}],
        label=label
    )
    # Enable the two traces we want to see
    for j in range(num_lines):
        step['args'][0]["visible"][num_lines*i+j] = True
    # Add step to step list
    steps.append(step)

sliders = [dict(
    active = 1,
    steps = steps,
    pad={"t": 80, "l":50, "r":50},
)]

fig.update_layout(
    legend=dict(traceorder="reversed"),
    font=dict(size=16),
    sliders = sliders,
    template="none",
)
fig.update_xaxes(
    range=[0,2],
    title_text="Temperature anomaly"
)
fig.update_yaxes(
    range=[0,1],
    title_text="Distorted probability"
)
fig.show()

Compute the implied drift distortion:

$$
h^* = - \cfrac{\frac{d\phi(y)}{dy} + \frac{(1-\eta)}{\delta}(\gamma_1 + \gamma_2 y) }{\xi_b}\sigma_y \tilde e
$$

The distortion is essentially constant as a function of the state variable and hence over time.
The drift distortion over time the four parameter configuration is as the following:

In [214]:
# drift distortion
fig = go.Figure()
for ξ_b_i, ξ_p_i in params_list:
    ht = solution[(ξ_b_i, ξ_p_i)]['simulation_res']['ht']
    fig.add_trace(go.Scatter(x=np.arange(0,100), y=ht, 
                             visible=False,
                             line=dict(width=2.5),
                             name=r'$\xi_b = {:.2f}, \xi_p = {:.2f}$'.format(ξ_b_i, ξ_p_i)
                            )) 



fig.data[-1].visible = True
steps = []
for i in range(len(params_list)):
    label = r'ξ_b = {:.2f}, ξ_p = {:.2f}'.format(params_list[i][0],params_list[i][1])
    step = dict(
        method ='update',  
        args = [{'visible': [False] * len(fig.data)},
               {"title": r'$Distorted\ probability\\ \xi_b = {:.2f},\quad \xi_p = {:.2f}$'.format( 
                   params_list[i][0],
                   params_list[i][1]
               )}],
        label=label
    )
    step['args'][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active = len(params_list)-1,
    steps = steps,
    pad={"t": 80, 'l':50, 'r':50},
)]

fig.update_layout(
    font=dict(size=16),
    sliders = sliders,
    template="none",
)
fig.update_xaxes(
    range=[0,100],
    title_text="Years"
)
fig.update_yaxes(
    range=[0,0.2],
    title_text="Drift distortion"
)

In [114]:
# emission trajectories
fig = go.Figure()
colors=["red", "darkgreen", "darkorange", "navy"]
for i in range(len(params_list)):
    ξ_b_i, ξ_p_i = params_list[i]
    et = solution[(ξ_b_i, ξ_p_i)]['simulation_res']['et']
    name = r"$\xi_b = {:.2f},\ \xi_p = {:.2f}$".format(ξ_b_i, ξ_p_i)
    if ξ_b_i == 100_000 and ξ_p_i == 100_000:
        name = "baseline"
    fig.add_trace(go.Scatter(x=np.arange(0,100), y=et, line=dict(color=colors[i],width=3), name=name)) 

fig.update_layout(
    title="Emission trajectories",
    legend=dict(x=0.01, y=0.01, font=dict(size=14), bgcolor='rgba(0,0,0,0)'),
    font=dict(size=16),
    template="none",
)
fig.update_xaxes(
    range=[0,88],
    title_text="Years"
)
fig.update_yaxes(
    range=[0,10],
    title_text="Emissions"
)

Social cost of carbon, expressed in logarithms, for $y \in [0, \bar y]$.

Compute as follows:

$$
\begin{aligned}
\log SCC_t =& \log(1000) + \log C_0 - \log N_t - \log \tilde{\mathcal{E}}_t + \log \eta - \log (1 - \eta)\\
=& \log(1000) + \log C_0 - \log (\gamma_1 Y_t + \frac{\gamma_2}{2} Y_t^2 ) - \log \tilde{\mathcal{E}}_t + \log \eta - \log (1 - \eta)
\end{aligned}
$$

Values of the paramters are as follows:

| Parameters | values |
| :---:| :---|
|$\eta$ | 0.032 | 
|$\gamma_1$ | 0.00017675 |
|$\gamma_2$ | 0.0044|
|Initial consumption, $C_0$| 17.39 |


In [124]:
# log SSC trajectories
fig = go.Figure()
C0 = 17.39
for i, [ξ_b_i, ξ_p_i] in enumerate(params_list):
    et = solution[(ξ_b_i, ξ_p_i)]['simulation_res']['et']
    yt = solution[(ξ_b_i, ξ_p_i)]['simulation_res']['yt']
    logSCCt = np.log(1000) + np.log(C0) - (γ_1*yt[et>0] + γ_2/2*yt[et>0]**2) - np.log(et[et>0]) + np.log(η) - np.log(1-η)
    name = r"$\xi_b = {:.2f},\ \xi_p = {:.2f}$".format(ξ_b_i, ξ_p_i)
    if ξ_b_i == 100_000 and ξ_p_i == 100_000:
        name = "baseline"
    fig.add_trace(go.Scatter(x=np.arange(0,100), y=logSCCt, line=dict(color=colors[i],width=3), name=name)) 

fig.update_layout(
    title=r'Social cost of carbon expressed in logarithms',
    legend=dict(
        x=0.01, y=1, 
        font=dict(size=14        itemsizing='constant'), 
        traceorder='reversed', 
        bgcolor='rgba(0,0,0,0)',
    ),
    font=dict(size=16),
    template="none",
)
fig.update_xaxes(
    range=[0,88],
    showline=True,
    linecolor='black',
    title_text="Years"
)
fig.update_yaxes(
    showline=True,
    linecolor='black',
    title_text=r'$\log SCC_t$'
)

## 7.2 Structured climate uncertainty

We go back to the $(\xi_a, \xi_b, \xi_p) = (0.01, \infty, 5)$ setting.

In [126]:
ξ_w = 100_000
ξ_a = 1/100
ξ_p = 5

# Prepare ϕ conditional on low, high, extreme damage
model_res_list = []
for γ_2p_i in γ_2p:
    model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p_i, θ_list, πc_o, ξ_w, ξ_a) 
    model_res = solve_hjb_y(y_grid_long, model_args, v0=None, ϵ=1.,
                            tol=1e-8, max_iter=5_000, print_iteration=False)
    model_res_list.append(model_res)

ϕ_list = [res['v'] for res in model_res_list]
certainty_equivalent = -ξ_p*np.log(np.average(np.exp(-1./ξ_p*np.array(ϕ_list)), axis=0, weights=πd_o))
# Change grid from 0-4 to 0-2
ϕ_i = np.array([temp[:n_bar] for temp in ϕ_list])
# Compute ϕ with jump (impose boundary condition)
model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p, θ_list, πc_o, ϕ_i, πd_o, ξ_w, ξ_p, ξ_a)
model_res = solve_hjb_y_jump(y_grid_short, model_args, 
                         v0=np.average(ϕ_i, weights=πd_o, axis=0),
                         ϵ=1., tol=1e-8, max_iter=5_000, print_iteration=False)
simulation_res = simulate_jump(model_res, θ_list)
res = dict(
    model_res=model_res, 
    simulation_res=simulation_res, 
    certainty_equivalent=certainty_equivalent
)

Converged. Total iteration 5000: LHS Error: 3.312423817458665e-05; RHS Error 8.65694637807446e-05
Converged. Total iteration 1507: LHS Error: 9.965619884866328e-09; RHS Error 0.00047002697358666107
Converged. Total iteration 1621: LHS Error: 9.95228432998374e-09; RHS Error 0.002278804686049074
Converged. Total iteration 443: LHS Error: 9.751046192718604e-09; RHS Error 0.0008702963234703057


In [212]:
# worst case probability
fig = go.Figure()
xbins = dict(start=0.8, end=3.0, size=0.15)
fig.add_trace(go.Histogram(
    x=θ_list*1000,
    histnorm='probability density',
    name='baseline',
    xbins=xbins,
    visible=True,
    marker = dict(color = "salmon", line = dict(color="darkgray", width = 1)),
    opacity=0.7
))

selected_years = [10, 50, 90]
for year in selected_years:
    fig.add_trace(go.Histogram(
        x=θ_list*1000,
        y=simulation_res["πct"][year]*np.exp(-year*δ),
        histnorm='probability density',
        histfunc='sum',
        xbins=xbins,
        marker = dict(color = "#1978d6", line = dict(color="gray", width = 1)),
        opacity=0.5,
        name="year {:d}".format(year),
        visible=False
    ))

fig.data[-1].visible = True

steps = []
for i in range(len(selected_years)):
    label = r'year = {:d}'.format(selected_years[i])
    step = dict(
        method ='update',  
        args = [{'visible': [False] * len(fig.data)},
               {"title": "title"}],
        label=label
    )
    step['args'][0]["visible"][0] = True
    step['args'][0]["visible"][i+1] = True
    # Add step to step list
    steps.append(step)

sliders = [dict(
    active = len(fig.data)-3,
    steps = steps,
    pad={"t": 80, 'l':50, 'r':50},
)]



fig.update_layout(
    sliders=sliders,
    barmode='overlay',
    width=900, height=600,
    font=dict(size=16),
    template="none",
)
fig.update_xaxes(
    range=[0.8,3],
    showline=True,
    linecolor='black',
    title_text='Climate sensitivity',
)
fig.update_yaxes(
    range=[0,1.4],
    title_text="Density",
    showline=True,
    linecolor='black',
)

Continue to the next section:

[Section 8: Uncertainty decomposition](sec8_UncertaintyDecomposition.ipynb)